# Code

Running this notebook end-to-end will reproduce the solution. Step by step guide is also provided. You can skip some long running steps by executing corresponding cells of `Download.ipynb` to download artifacts.

In [4]:
import sys
import os
import glob
import yaml

In [5]:
!cat config.yaml

with open('config.yaml') as f:
    CONFIG = yaml.safe_load(f)
    
BASE_PATH = CONFIG['base_path']
CONFIG_PATH = os.path.join(BASE_PATH, 'config.yaml')
RAPIDS_ENV = os.path.join(BASE_PATH, CONFIG['rapids-env'])
PYTORCH_ENV = os.path.join(BASE_PATH, CONFIG['pytorch-env'])

base_path: ./ # working dir
# environments
rapids-env: rapids-env/bin/python
pytorch-env: pytorch-env/bin/python
# artifacts paths
embeds_path: embeds # path to embeddings 
models_path: models # store the models
helpers_path: helpers # store reformated datasets
temporal_path: temporal # store external data from FTP (temporal because different report dates are used)


base_models: # all models and postprocessing path
    pb_t5esm4500_raw:
        embeds: 
            - t5
            - esm_small
        conditional: false
        bp: 3000
        mf: 1000
        cc: 500
        
    pb_t5esm4500_cond:
        embeds: 
            - t5
            - esm_small
        conditional: true
        bp: 3000
        mf: 1000
        cc: 500
        
    pb_t54500_raw:
        embeds: 
            - t5
        conditional: false
        bp: 3000
        mf: 1000
        cc: 500
        
    pb_t54500_cond:
        embeds: 
            - t5
        conditional: true
        bp: 3000
        mf: 

# 1. Preparation

### 1.1. Setup envs

Create the following python envs:

* `pytorch-env` - env to deal with all DL models
* `rapids-env`  - env to preprocess via RAPIDS and train py-boost and logregs

In [6]:
!./create-rapids-env.sh {BASE_PATH}
!./create-pytorch-env.sh {BASE_PATH}


EnvironmentLocationNotFound: Not a conda environment: /home/anton/CAFA5-protein-function-prediction-2nd-place

conda 23.5.2
Retrieving notices: ...working... done
Channels:
 - rapidsai
 - conda-forge
 - nvidia
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/anton/CAFA5-protein-function-prediction-2nd-place/rapids-env

  added / updated specs:
    - cuda-version=11.2
    - python=3.8
    - rapids=23.02


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    c-ares-1.25.0              |       hd590300_0         153 KB  conda-forge
    exceptiongroup-1.2.0       |     pyhd8ed1ab_2          20 KB  conda-forge
    fastavro-1.9.3             |   py38h01eb140_0         504 KB  conda-forge
    fonttools-4.47.2           |   py38h01eb140_0         2.2 MB  conda-forge
    jinja2-3.1.3               |     pyhd8ed1ab_0         109 KB  conda-forg

exceptiongroup-1.2.0 | 20 KB     |                                       |   0% 
jupyter_server-2.12. | 313 KB    |                                       |   0% 

jinja2-3.1.3         | 109 KB    |                                       |   0% 


fonttools-4.47.2     | 2.2 MB    |                                       |   0% 



markdown-3.5.2       | 75 KB     |                                       |   0% 




rpds-py-0.17.1       | 994 KB    |                                       |   0% 





wcwidth-0.2.13       | 32 KB     |                                       |   0% 






lz4-4.3.3            | 36 KB     |                                       |   0% 







jupyter_core-5.7.1   | 77 KB     |                                       |   0% 








fastavro-1.9.3       | 504 KB    |                                       |   0% 









nbconvert-core-7.14. | 183 KB    |                                       |   0% 










exceptiongroup-1.2.0 | 20 KB     | ###################

Using cached py_boost-0.4.3-py3-none-any.whl (58 kB)
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.41.1
    Uninstalling llvmlite-0.41.1:
      Successfully uninstalled llvmlite-0.41.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.2.0 requires cupy-cuda11x, which is not installed.
cudf-kafka 23.2.0 requires cython, which is not installed.
cugraph 23.2.0+0.g450c25b8.dirty requires cupy-cuda11x, which is not installed.
cuml 23.2.0 requires seaborn, which is not installed.
dask-cudf 23.2.0 requires cupy-cuda11x, which is not installed.
cudf 23.2.0 requires protobuf==4.21, but you have protobuf 4.21.12 which is incompatible.
cudf 23.2.0 requires pyarrow==10, but you have pyarrow 10.0.1 which is incompatible.
/home/anton/CAFA5-protein-function-prediction-2nd-place

EnvironmentLocationNotFound: Not a conda environment: /home/a

Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Channels:
 - conda-forge
 - defaults
 - nvidia
 - pytorch
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/anton/CAFA5-protein-function-prediction-2nd-place/pytorch-env

  added / updated specs:
    - cupy


The following NEW packages will be INSTALLED:

  cuda-version       conda-forge/noarch::cuda-version-10.2-h4767cc1_2 
  cudatoolkit        conda-forge/linux-64::cudatoolkit-10.2.89-h713d32c_10 
  cupy               conda-forge/linux-64::cupy-12.1.0-py39h98de0c3_0 
  fastrlock          conda-forge/linux-64::fastrlock-0.8-py39h5a03fae_2 
  python_abi         conda-forge/linux-64::python_abi-3.9-2_cp39 

The following packages will be SUPERSEDED by a higher-priority channel:

  _libgcc_mutex                                   pkgs/main --> conda-forge 
  certifi            pkgs/main/linux-64::certifi-2023.11.1~ --> conda-forge/noarch::certifi-2023.11.17-pyh

  Using cached safetensors-0.4.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached lightning_utilities-0.10.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached fsspec-2023.12.2-py3-none-any.whl.metadata (6.8 kB)
  Using cached decorator-5.1.1-py3-none-any.whl (9.1 kB)
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached matplotlib_inline-0.1.6-py3-none-any.whl (9.4 kB)
  Using cached prompt_toolkit-3.0.43-py3-none-any.whl.metadata (6.5 kB)
  Using cached pygments-2.17.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached stack_data-0.6.3-py3-none-any.whl.metadata (18 kB)
  Using cached traitlets-5.14.1-py3-none-any.whl.metadata (10 kB)
  Using cached exceptiongroup-1.2.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached pexpect-4.9.0-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached parso-0.8.3-py2.py3-none-any.whl (100 kB)
  Using cached ptyprocess-0.7.0-py2.py3-none-any.whl (13 kB)
  Using cached wcwidth-0.2.13-py2.

### 1.2. Get the input data

Here we describe what should be stored in the working dir to reproduce the results

Following data scheme was provided by Kaggle:

    ./Train - cafa train data
    ./Test (targets) - cafa test data
    ./sample_submission.tsv - cafa ssub
    ./IA.txt - cafa IA

    
Following are the solution code libraries, scipts, and notebooks used for training:

    ./protlib
    ./protnn
    ./nn_solution
    
And the installed envs

    ./pytorch-env
    ./rapids-env

### 1.3. Produce the helpers data

First, we made some preprocessing of the input data to store everything in format that is convinient to us to handle and manipulate. Here is the structure:

    ./helpers
        ./fasta - fasta files stored as feather
            ./train_seq.feather
            ./test_seq.feather
        ./real_targets - targets stored as n_proteins x n_terms parquet containing 0/1/NaN values
            ./biological_process
                ./part_0.parquet
                ...
                ./part_14.parquet
                ./nulls.pkl - NaN rate of each term
                ./priors.pkl - prior mean of each term (excluding NaN cells, like np.nanmean)
            ./cellular_component
            ./molecular_function
            

In [7]:
%%time
# parse fasta files and save as feather
!{RAPIDS_ENV} protlib/scripts/parse_fasta.py \
    --config-path {CONFIG_PATH}

# convert targets to parquet and calculate priors
!{RAPIDS_ENV} protlib/scripts/create_helpers.py \
    --config-path {CONFIG_PATH} \
    --batch-size 10000

1524980it [00:02, 750164.78it/s]
1339949it [00:00, 1637363.93it/s]
/home/anton/CAFA5-protein-function-prediction-2nd-place
15it [21:02, 84.19s/it]
CPU times: user 11.5 s, sys: 4.42 s, total: 15.9 s
Wall time: 23min 3s


### 1.4. Get external data

Datasets downloaded from outside and then processed. First step is downloading and parsing the datasets. After parsing, script will separate the datasets by the evidence codes. The most important split for us is kaggle/no-kaggle split. We refer `kaggle` as experimental codes, `no-kaggle` as electornic labeling, that will be used as features for the stacker models. Downloading takes quite a long time, while processing takes about 1 hour. The required structure after execution

    ./temporal - extra data downloaded from http://ftp.ebi.ac.uk/pub/databases/GO/goa/old/UNIPROT/
    ./labels   - extracted and propagated labeling
        ./prop_test_leak_no_dup.tsv - leakage labeling
        ./prop_test_no_kaggle.tsv   - electronic labels test
        ./prop_train_no_kaggle.tsv  - electronic labels train
        
    ./cafa-terms-diff.tsv - reproduced difference between ML's dataset and our parsed labels
    ./prop_quickgo51.tsv  - reproduced MT's quickgo 37 proteins
    
    
Other files are temporary and not needed for future work

In [ ]:
# download external data from ebi.ac.uk
!{RAPIDS_ENV} protlib/scripts/downloads/dw_goant.py \
    --config-path {CONFIG_PATH}

# # parse the files
!{RAPIDS_ENV} protlib/scripts/parse_go_single.py \
    --file goa_uniprot_all.gaf.216.gz \
    --config-path {CONFIG_PATH}

!{RAPIDS_ENV} protlib/scripts/parse_go_single.py \
    --file goa_uniprot_all.gaf.214.gz \
    --config-path {CONFIG_PATH} \
    --output old214

The next step is propagation. Since ebi.ac datasets contains the labeling without propagation, we will apply the rules provided in organizer's repo to labeling more terms. We will do it only for `goa_uniprot_all.gaf.216.gz` datasets since it is the actual dataset at the active competition phase

In [ ]:
folder = BASE_PATH + '/temporal'

for file in glob.glob(folder + '/labels/train*') + glob.glob(folder + '/labels/test*'):
    name = folder + '/labels/prop_' + file.split('/')[-1]

    !{RAPIDS_ENV} {BASE_PATH}/protlib/scripts/prop_tsv.py \
        --path {file} \
        --graph {BASE_PATH}/Train/go-basic.obo \
        --output {name} \
        --device 0 \
        --batch_size 30000 \
        --batch_inner 5000

The last part is reproducing MT's datasets that are commonly used in all public kernels. We didn't use it directly, but we used `cafa-terms-diff` dataset, that represents the difference between our labeling obtained by parsing `goa_uniprot_all.gaf.216.gz` dataset and `all_dict.pkl` dataset given by MT. As he claims in the dicussion [here](https://www.kaggle.com/competitions/cafa-5-protein-function-prediction/discussion/404853#2329935) he used the same FTP source as we. But our source is more actual than the public. So the difference is actually the temporal. After analysis, we find out, that we are able to reproduce it as the difference between `goa_uniprot_all.gaf.216.gz` and `goa_uniprot_all.gaf.214.gz` sources. So, we just create `cafa-terms-diff` dataset by the given script. The only difference between the source in the kaggle script and used here is deduplication. We removed duplicated protein/terms pairs from the dataset, it has almost zero impact on the metric value (less than 1e-4)


In [ ]:
# create datasets
!{RAPIDS_ENV} {BASE_PATH}/protlib/scripts/reproduce_mt.py \
    --path {BASE_PATH}/temporal \
    --graph {BASE_PATH}/Train/go-basic.obo

# # make propagation for quickgo51.tsv
!{RAPIDS_ENV} {BASE_PATH}/protlib/scripts/prop_tsv.py \
    --path {BASE_PATH}/temporal/quickgo51.tsv \
    --graph {BASE_PATH}/Train/go-basic.obo \
    --output {BASE_PATH}/temporal/prop_quickgo51.tsv \
    --device 0 \
    --batch_size 30000 \
    --batch_inner 5000

### 1.5 Preparation step for neural networks

Produce some helpers to train NN model. Creates the following data:

    ./helpers/feats
        ./train_ids_cut43k.npy
        ./Y_31466_labels.npy
        ./Y_31466_sparse_float32.npz

In [8]:
%%time

!{PYTORCH_ENV} {BASE_PATH}/nn_solution/prepare.py \
    --config-path {CONFIG_PATH}

(5363863, 3)
GO:0005575    92912
GO:0008150    92210
GO:0110165    91286
GO:0003674    78637
GO:0005622    70785
              ...  
GO:0031772        1
GO:0042324        1
GO:0031771        1
GO:0051041        1
GO:0102628        1
Name: term, Length: 31466, dtype: int64
aspect
BPO    21285
CCO     2957
MFO     7224
Name: term, dtype: int64
CPU times: user 104 ms, sys: 28.1 ms, total: 132 ms
Wall time: 10.4 s


# 2. Embeddings

In [9]:
!mkdir embeds

mkdir: cannot create directory ‘embeds’: File exists


### 2.1 T5 pretrained inference

    ./embeds
        ./t5
            ./train_ids.npy
            ./train_embeds.npy
            ./test_ids.npy
            ./test_embeds.npy

In [ ]:
%%time
!{PYTORCH_ENV} {BASE_PATH}/nn_solution/t5.py \
    --config-path {CONFIG_PATH} \
    --device 0

### 2.2 ESM pretrained inference

    ./embeds
        ./esm_small
            ./train_ids.npy
            ./train_embeds.npy
            ./test_ids.npy
            ./test_embeds.npy

In [ ]:
%%time
!{PYTORCH_ENV} {BASE_PATH}/nn_solution/esm2sm.py \
    --config-path {CONFIG_PATH} \
    --device 0

# 3. Base models

In [10]:
!mkdir models

mkdir: cannot create directory ‘models’: File exists


### 3.1. Train and inference py-boost models

GBDT models description:

1) Features: T5 + taxon, targets: multilabel

2) Features: T5 + taxon, targets: conditional

3) Features: T5 + ESM + taxon, targets: multilabel

4) Features: T5 + ESM + taxon, targets: conditional

Pipeline and hyperparameters are the same for all the models. Target is 4500 output: BP 3000, MF: 1000, CC: 500. All models could be ran in parallel to save a time. We used single V100 32GB and it requires about 15 hours to train 5 fold CV loop for each model type. 32GB GPU RAM is required, otherwise OOM will occur. Structure is:
    
    ./models
        ./pb_t54500_raw
            ./models_0.pkl
            ...
            ./models_4.pkl
            ./oof_pred.pkl
            ./test_pred.pkl
        ./pb_t54500_cond
            ...
        ./pb_t5esm4500_raw
            ...
        ./pb_t5esm4500_cond
            ...

In [15]:
for model_name in ['pb_t54500_raw', 'pb_t54500_cond', 'pb_t5esm4500_raw', 'pb_t5esm4500_cond', ]:

    print(f'Training {model_name}')

    !{RAPIDS_ENV} {BASE_PATH}/protlib/scripts/train_pb.py \
        --config-path {CONFIG_PATH} \
        --model-name {model_name} \
        --device 0


Training pb_t54500_raw
True
trg filled
trg filled
trg filled
(142246, 1056) (141865, 1056)
(113769,) (28477,)
[23:01:58] Stdout logging level is INFO.
[23:01:58] GDBT train starts. Max iter 20000, early stopping rounds 300
[23:02:26] Iter 0; Sample 0, BCE = 0.03245292768610193; 
[23:03:09] Iter 100; Sample 0, BCE = 0.02795873129372635; 
Training pb_t54500_cond
Training pb_t5esm4500_raw


UnboundLocalError: local variable 'child' referenced before assignment

### 3.2. Train and inference logreg models

Logistic Regression models description:

1) Features: T5 + taxon, targets: multilabel

2) Features: T5 + taxon, targets: conditional


Pipeline and hyperparameters are the same for all the models. Target is 13500 output: BP 10000, MF: 2000, CC: 1500. All models could be ran in parallel to save a time. We used single V100 32GB and it requires about 10 hours for model 1 and 2 hours for model 2 to train 5 fold CV loop. 32GB GPU RAM is required, otherwise OOM will occur. Structure is:

    ./helpers
        ./folds_gkf.npy
    ./models
        ./lin_t5_raw
            ./models_0.pkl
            ...
            ./models_4.pkl
            ./oof_pred.pkl
            ./test_pred.pkl
        ./lin_t5_cond
            ...

In [ ]:
for model_name in ['lin_t5_raw', 'lin_t54500_cond']:

    print(f'Training {model_name}')

    !{RAPIDS_ENV} {BASE_PATH}/protlib/scripts/train_lin.py \
        --config-path {CONFIG_PATH} \
        --model-name {model_name} \
        --device 0


### 3.3. Train and inference NN models

Structure is:

    ./models
        ./nn_serg
            ./model_0_0.pt
            ...
            ./model_11_4.pt
            ./pytorch-keras-etc-3-blend-cafa-metric-etc.pkl 

In [ ]:
# first, create train folds (the same as used for pb_t54500_cond model)
!{RAPIDS_ENV} {BASE_PATH}/protlib/scripts/create_gkf.py \
    --config-path {CONFIG_PATH}

# train models
!{PYTORCH_ENV} {BASE_PATH}/nn_solution/train_models.py \
    --config-path {CONFIG_PATH} \
    --device 0

# inference models
!{PYTORCH_ENV} {BASE_PATH}/nn_solution/inference_models.py \
    --config-path {CONFIG_PATH} \
    --device 0

# reformat to use in stack
!{PYTORCH_ENV} {BASE_PATH}/nn_solution/make_pkl.py \
    --config-path {CONFIG_PATH}

# 4. Final model

### 4.1. Train GCN models

This step is training 3 independent stacking models for each ontology. Models are trained on single V100 GPU and it takes about 13 hours for BP, 4 hours for MF and 2 hours for CC. 32 GB GPU RAM is required to fit. Could be trained in parallel if 2 GPUs are avaliable - BP and MF/CC. Structure:

    ./models
        ./gcn
            ./bp
                ./checkpoint.pth
            ./mf
                ./checkpoint.pth
            ./cc
                ./checkpoint.pth
                

In [ ]:
%%time

for ont in ['bp', 'mf', 'cc']:
    !{PYTORCH_ENV} {BASE_PATH}/protnn/scripts/train_gcn.py \
        --config-path {CONFIG_PATH} \
        --ontology {ont} \
        --device 0

### 4.2. Inference GCN models and TTA

Inference and Test-Time-Augmentation. Structure:

    ./models
        ./gcn
            ./pred_tta_0.tsv
            ...
            ./pred_tta_3.tsv


In [ ]:
%%time

!{PYTORCH_ENV} {BASE_PATH}/protnn/scripts/predict_gcn.py \
    --config-path {CONFIG_PATH} \
    --device 0

### 4.3. Postprocessing and build submission file

Here we do the following:

1) Average TTA predictions
2) Perform min prop
3) Perform max prop
4) Average min/max prop steps, add external leakage data and make submission

Structure:

    ./models
        ./postproc
            ./pred.tsv     - avg TTA
            ./pred_min.tsv - min prop
            ./pred_max.tsv - max prop
            
    ./sub
        ./submission.tsv   - final results

In [ ]:
# since we have 4 TTA predictions, we need to aggregate all as an average
!{RAPIDS_ENV} {BASE_PATH}/protlib/scripts/postproc/collect_ttas.py \
    --config-path {CONFIG_PATH} \
    --device 0

# create 0.3 * pred + 0.7 * max children propagation
!{RAPIDS_ENV} {BASE_PATH}/protlib/scripts/postproc/step.py \
    --config-path {CONFIG_PATH} \
    --device 0 \
    --batch_size 30000 \
    --batch_inner 3000 \
    --lr 0.7 \
    --direction min

# create 0.3 * pred + 0.7 * min parents propagation
!{RAPIDS_ENV} {BASE_PATH}/protlib/scripts/postproc/step.py \
    --config-path {CONFIG_PATH} \
    --device 0 \
    --batch_size 30000 \
    --batch_inner 3000 \
    --lr 0.7 \
    --direction max

# here we average min prop and max prop solutions, mix with cafa-terms-diff and quickgo51 datasets from 1.4
!{RAPIDS_ENV} {BASE_PATH}/protlib/scripts/postproc/make_submission.py \
    --config-path {CONFIG_PATH} \
    --device 0 \
    --max-rate 0.5

# Result

Result is stored in `./sub/submission.tsv`

In [ ]:
!head {BASE_PATH}/sub/submission.tsv